In [2]:
import yfinance as yf
import pandas as pd  
from datetime import datetime, timedelta
import numpy as np  
# Make imports work 

Define the list of tickers

In [3]:
tickers = ['LMT' , 'GM' , 'RUT' , 'NVDA' , 'MSFT']
#choose tickers for optimal portfolio gains. Considered industrial implications and past financial performance , stock valuation

Setting the date to 5 years prior

In [4]:
end_date = datetime.today()

In [5]:
start_date = end_date - timedelta(days = 5*365)
print(start_date)

2020-02-17 21:01:48.840245


Create an empty dataframe  to collect closing prices and adjust for all variables

In [6]:
adj_close_df = pd.DataFrame()

Downloading the close prices for each ticker

In [7]:
tickers = ['LMT' , 'GM' , 'RUT' , 'NVDA' , 'MSFT']
for ticker in tickers:
    data = yf.download(ticker, start= start_date, end = end_date)
    adj_close_df[ticker] = data['Adj Close']

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['RUT']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2020-02-17 21:01:48.840245 -> 2025-02-15 21:01:48.840245)')
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [8]:
print(adj_close_df)

                   LMT         GM  RUT        NVDA        MSFT
Date                                                          
2020-02-18  381.045258  33.139412  NaN    7.383298  179.082779
2020-02-19  380.109253  33.630581  NaN    7.834654  179.619858
2020-02-20  372.664062  33.986916  NaN    7.685283  176.876846
2020-02-21  374.265167  33.370556  NaN    7.321058  171.285278
2020-02-24  366.872589  31.868156  NaN    6.803476  163.900238
...                ...        ...  ...         ...         ...
2025-02-10  448.220001  46.570000  NaN  133.570007  412.220001
2025-02-11  449.359985  46.700001  NaN  132.800003  411.440002
2025-02-12  441.970001  47.689999  NaN  131.139999  409.040009
2025-02-13  434.720001  47.889999  NaN  135.289993  410.540009
2025-02-14  423.190002  48.369999  NaN  138.850006  408.429993

[1257 rows x 5 columns]


Calculating Lognormal returns 

In [9]:
log_returns = np.log(adj_close_df/adj_close_df.shift(1))
log_returns = log_returns.dropna()

CALCULATING COVARIANCE MATRIX (Use 252 trading days)

In [10]:
cov_matrix = log_returns.cov()*252
print(cov_matrix)
#displays covariance matrix with relative stock/index prices

      LMT  GM  RUT  NVDA  MSFT
LMT   NaN NaN  NaN   NaN   NaN
GM    NaN NaN  NaN   NaN   NaN
RUT   NaN NaN  NaN   NaN   NaN
NVDA  NaN NaN  NaN   NaN   NaN
MSFT  NaN NaN  NaN   NaN   NaN


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pandas/core/frame.py:11211: RuntimeWarning: Degrees of freedom <= 0 for slice
  base_cov = np.cov(mat.T, ddof=ddof)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


STANDARD DEVIATION

In [11]:
def standard_deviation(weights, cov_matrix):
    variance = weights.T @ cov_matrix @ weights
    return np.sqrt(variance)

EXPECTED RETURN

In [12]:
def expected_return(weights, log_return):
    return np.sum(log_returns.mean(*weights))*252
# average return calcuulated based on lognormal return and weights

CALCULATING SHARPE RATIO 

In [13]:
def sharpe_ratio(weights, log_returns, cov_matrix, risk_free_rate):
    return(expected_return (weights, log_returns)- risk_free_rate) / standard_deviation(weights, cov_matrix)

CALCULATING RISK FREE RATE 
(using fred API to calculate average treasury bond rate
also can take standard assumption of 2%)

In [14]:
from fredapi import Fred

fred = Fred(api_key= '2d60db8717ba3d033f961ac04cf4855e')
ten_year_treasury_rate = fred.get_series_latest_release('GS10' / 100)

#Set the risk free rate

risk_free_rate = ten_year_treasury_rate.iloc[-1]
print(risk_free_rate)

TypeError: unsupported operand type(s) for /: 'str' and 'int'

Function to minimize (Negative sharp ratio )

In [ ]:
def neg_sharpe_ratio(weights, log_returns, cov_matrix, risk_free_rate):
    return -sharpe_ratio(weights, log_returns, cov_matrix, risk_free_rate)

Setting constraints: ensuring all portfolio weights are equal to one. Setting lower and upper bounds for the ratio of investments in multiple stocks. The constraint variable is set as a dictionary

In [ ]:
constraints = {'type': 'eq', 'fun': lambda weights: np.sum(weights) -1}
bounds = [(0, 0.5) for_in range(len(tickers))]

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1285108266.py, line 2)

In [ ]:
initial_weights = np.array([1/len(tickers)]*len(tickers))

Optimizing weights to the maximum sharpe ratio 

In [ ]:
optimized_results = minimize(neg_sharpe_ratio, initial_weights, args=(log_returns, cov_matrix, risk_free_rate), method= 'SLSQP', constraints=constraints, bounds=bounds)

# args are the arguements and constraints are used for optimal weights extraction

In [ ]:
optimal_weights = optimized_results.x

Data Visualisation of efficent frontiers/ weights

In [ ]:
print("optimal wights:")
for ticker, weight in zip(tickers, optimal_weights):
   print(f"{ticker}: {weight: .4f}") #understand print statement 


optimal_portfolio_return = expected_return(optimal_weights, log_returns)
optimal_portfolio_vollatility = standard_deviation(optimal_weights, cov_matrix)
optimal_sharpe_ratio = sharpe_ratio(optimal_weights, log_returns, cov_matrix, risk_free_rate)

print(f"Expected Annual Return: {optimal_portfolio_return:.4f}")
print(f"Expected volatility: {optimal_portfolio_vollatility:.4f}")
print(f"Sharpe Ratio: {optimal_sharpe_ratio:.4f}")


Final portfolio weightage plot

In [ ]:
import matplotlib.pyplot as plt

#creating a bar chart 

plt.figure(figsize=(10, 6)) #understnad figsize
plt.bar(tickers, optimal_weights)

#Add labels
plt.xlabel('Assets')
plt.ylabel('Optimal Weights')
plt.title('Optima portfolio Weights')

#Chart Display
plt.show()

MONTE CARLO SIMULATIONS

In [ ]:
#simulation imports 
from pandas_datareader import data as pdr


def get_data(stocks, start, end):
    StockData = pdr.get_data_yahoo(stocks, start, end)
    StockData = StockData['Close']
    returns = StockData.pct.change()
    mean_returns = returns.mean()
    cov_matrix = returns.cov()
    return mean_returns, cov_matrix

stock_list = ['LMT' , 'GM' , 'RUT' , 'NVDA' , 'MSFT']
end_date = datetime.today()
start_date = end_date - timedelta(days = 5*365)

weights = optimal_weights
print(weights)


In [ ]:
# Set MC simulations

mc_sims = 100
T = 100

Mean_matrix = np.full(shape=(T, len(weights)), fill_value = mean_returns) # setting matrix of return vectors
Mean_matrix = Mean_matrix.T #(transpose of the matrix)

Portfolio_sims = np.full(shape = (T, mc_sims), fill_value=0.0)

initial_value = 1000

for m in range(0, mc_sims):
    Z = np.random.normal(size=(T, len(weights)))
    L = np.linalg.cholesky(cov_matrix)
    Mc_return = Mean_matrix + np.inner(L, Z) #Using cholesky decomposition
    Portfolio_sims[: ,m] = np.cumprod(np.inner(weights, Mc_return.T)+1)*initial_value

    

MC visulaisation

In [ ]:
plt.plot(Portfolio_sims)
plt.ylabel('Value in $')
plt.xlabel('Days')
plt.title('MC simulation of a portfolio')
plt.show()